In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import skmob
from skmob import preprocessing
from shapely.geometry import LineString
import folium

In [2]:
def create_plot(data):
    # Ambil baris pertama dari DataFrame
    first_row = data.iloc[0]

    # Ambil nilai latitude dan longitude dari baris pertama
    latitude = first_row['latitude']
    longitude = first_row['longitude']
    
    m = folium.Map(location=[latitude, longitude], zoom_start=25)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [3]:
def create_pivot(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df_pivot = df['maid'].groupby(df['timestamp'].dt.date).value_counts()
    pivot = df_pivot.unstack().fillna(0).astype(int)

    total_counts = pivot.sum(axis=0)
    sorted_columns = total_counts.sort_values(ascending=False).index
    pivot_sorted = pivot[sorted_columns]
    return pivot_sorted

In [4]:
def create_line(data):
    data = data.to_crs('EPSG:4326')
    data = data['geometry']
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    avg_lat = data.apply(lambda x: x.centroid.y).mean()
    avg_lon = data.apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

    # Tambahkan polyline untuk setiap linestring
    for linestring in data:
        coordinates = [(lat, lon) for lon, lat in linestring.coords]
        folium.PolyLine(locations=coordinates, color='purple', weight=8, opacity=0.7).add_to(m)

    # Tambahkan GeoJson dari data GeoPandas
    folium.GeoJson(data.to_json(), name='Garis Jalan').add_to(m)

    return m

## Import data

In [5]:
path = './filter2_malar_jan.csv'
gps = pd.read_csv(path)

In [6]:
road = './Malioboro_around/Around_Malioboro_line.shp'
area_shape = gpd.read_file(road)

In [7]:
area_shape['highway'].unique()

array(['living_street', 'footway', 'residential', 'service', 'primary',
       'primary_link', 'steps', 'tertiary', 'path', 'secondary',
       'pedestrian', 'unclassified', 'secondary_link', 'tertiary_link',
       'cycleway', 'track'], dtype=object)

- living_street: Jalan yang didesain untuk mengutamakan pejalan kaki dan mengurangi kecepatan kendaraan.
- footway: Jalan khusus untuk pejalan kaki, biasanya terletak di kawasan pejalan kaki atau area non-kendaraan.
- residential: Jalan yang terletak di area perumahan atau pemukiman.
- service: Jalan yang digunakan untuk akses ke properti, seperti akses ke parkir atau bangunan bisnis.
- primary: Jalan utama dalam jaringan jalan yang memiliki volume lalu lintas tinggi.
- primary_link: Bagian pendukung dari jalan primer yang menyediakan akses ke atau dari jalan primer.
- steps: Tangga untuk akses pejalan kaki yang terdiri dari serangkaian langkah.
- tertiary: Jalan yang memiliki volume lalu lintas lebih rendah dari jalan primer dan sekunder.
- path: Jalan yang dirancang untuk pejalan kaki, sepeda, atau kendaraan non-motor.
- secondary: Jalan yang memiliki volume lalu lintas di atas jalan tertier, tetapi di bawah jalan primer.
- pedestrian: Area yang ditujukan khusus untuk pejalan kaki, biasanya tanpa kendaraan bermotor.
- unclassified: Jalan yang tidak memiliki klasifikasi resmi, sering kali digunakan di kawasan perkotaan kecil.
- secondary_link: Bagian pendukung dari jalan sekunder yang menyediakan akses ke atau dari jalan sekunder.
- tertiary_link: Bagian pendukung dari jalan tertier yang menyediakan akses ke atau dari jalan tertier.
- cycleway: Jalan yang dirancang khusus untuk pengguna sepeda.
- track: Jalan yang digunakan oleh kendaraan, biasanya di luar jalan aspal, seperti jalan tanah atau jalan setapak.

In [8]:
highway = ['living_street','residential','primary','primary_link','tertiary', 'secondary', 'unclassified', 'secondary_link', 'tertiary_link','track']
area_vehicle = area_shape[area_shape['highway'].isin(highway)]
area_vehicle = area_vehicle[['osm_id','oneway','name', 'highway', 'geometry']]
area_vehicle.reset_index(inplace=True, drop=True)

In [42]:
create_line(area_vehicle)

In [55]:
# create_line(area_shape)

## Try to filtering

In [9]:
gps.head()

,maid,latitude,longitude,timestamp
0,0000370f-9b66-4bbf-a9a7-45b17876bc6b,-7.803500,110.364601,2022-01-04 12:12:14
1,0000370f-9b66-4bbf-a9a7-45b17876bc6b,-7.803500,110.364601,2022-01-04 12:12:23
2,00007d1d-2f96-4ed8-b169-441e79494e22,-7.814539,110.362915,2022-01-20 06:20:58
3,00007d1d-2f96-4ed8-b169-441e79494e22,-7.808829,110.367218,2022-01-20 06:37:13
4,00007d1d-2f96-4ed8-b169-441e79494e22,-7.810627,110.377083,2022-01-24 06:19:12


In [10]:
area_vehicle.head()

,osm_id,oneway,name,highway,geometry
0,1124471552,None,None,living_street,"LINESTRING (110.36030 -7.79281, 110.36025 -7.7..."
1,1130748031,yes,Jalan Alun Alun Utara,residential,"LINESTRING (110.36318 -7.80502, 110.36322 -7.8..."
2,1124296452,None,Sosrowijayan Wetan,living_street,"LINESTRING (110.36560 -7.79170, 110.36563 -7.7..."
3,724330070,None,None,residential,"LINESTRING (110.36895 -7.80552, 110.36868 -7.8..."
4,561138367,yes,Jalan Magelang,primary,"LINESTRING (110.36110 -7.77677, 110.36107 -7.7..."


In [11]:
# create_pivot(gps)

In [12]:
# orang = gps[(gps['maid'] == '9a63fd0e-d363-478a-8136-99b1eb82693f') & (gps['timestamp'] > '2021-12-25')]
# len(orang)

In [13]:
def filter_area_around_road(gps_data, road_data, distance):
    # Membuat GeoDataFrame dari data GPS
    gdf_gps = gpd.GeoDataFrame(
        gps_data,
        geometry=gpd.points_from_xy(gps_data.longitude, gps_data.latitude),
        crs="EPSG:4326"  # WGS84 coordinate reference system
    )

    # Membuat GeoDataFrame dari data jaringan jalan
    gdf_jalan = gpd.GeoDataFrame(
        road_data,
        geometry=road_data.geometry,
        crs="EPSG:4326"  # WGS84 coordinate reference system
    )

    # Mengubah sistem referensi koordinat jaringan jalan ke UTM (misalnya UTM zone 48S untuk Yogyakarta)
    gdf_jalan = gdf_jalan.to_crs("EPSG:32748")  # UTM zone 48S

    # Buffer jaringan jalan sebesar 20 meter
    buffered_jalan = gdf_jalan.buffer(distance)

    # Gabungkan semua buffered jalan menjadi satu geometri
    merged_buffer = buffered_jalan.unary_union

    # Mengubah sistem referensi koordinat data GPS ke UTM
    gdf_gps = gdf_gps.to_crs("EPSG:32748")  # UTM zone 48S

    # Membuat kolom baru untuk menyimpan jarak terdekat ke jalan
    gdf_gps['closest_distance_to_road'] = gdf_gps.geometry.apply(lambda x: x.distance(merged_buffer))

    # Mengambil data point yang jaraknya maksimal 20 meter dari jaringan jalan
    data_point_terdekat = gdf_gps[gdf_gps['closest_distance_to_road'] <= distance]

    return data_point_terdekat

In [14]:
df_filtered_20m = filter_area_around_road(gps, area_vehicle, 20)

In [15]:
len(gps) - len(df_filtered_20m)

384531

### Hapus data yg memiliki jarak 1 jam dari record sebelumnya dan record setelahnya

In [16]:
def remove_records_minutes(gps, timedelta):
    gps['timestamp'] = pd.to_datetime(gps['timestamp'])

    # Membuat dataframe sementara untuk menampung records yg memenuhi syarat
    df_filtered = pd.DataFrame()
    
    # Iterasi melalui setiap kelompok 'maid'
    for maid, group in gps.groupby('maid'):
        # Hitung perbedaan waktu antara setiap baris dengan baris sebelumnya dan baris setelahnya
        diff_prev = group['timestamp'] - group['timestamp'].shift(1)
        diff_next = group['timestamp'].shift(-1) - group['timestamp']
        
        # Aturan untuk hanya mengambil yg memenuhi syarat -> jarak kedepan/kebelakang < 60 menit
        to_keep = ((diff_prev <= pd.Timedelta(minutes=timedelta)) | (diff_next <= pd.Timedelta(minutes=timedelta)))

        # Ambil baris-baris yang ditandai untuk dipertahankan
        group_filtered = group[to_keep]

        # Tambahkan hasil filter untuk kelompok ini ke DataFrame akhir
        df_filtered = pd.concat([df_filtered, group_filtered]) 
        
    df_filtered.reset_index(inplace=True, drop=True)
    
    return df_filtered

In [17]:
df_filtered_20m_60min = remove_records_minutes(df_filtered_20m, 60)

In [18]:
len(df_filtered_20m) - len(df_filtered_20m_60min)

64904

In [20]:
df_filtered_20m_60min = df_filtered_20m_60min[['maid','latitude','longitude','timestamp']]
df_filtered_20m_60min.to_csv('filter3_20m_60min_malar_jan.csv', index=False)